In [1]:
import pandas as pd
from statistics import mean,stdev
import numpy as np
from scipy.spatial import distance
import math
import random
from scipy import linalg
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import adjusted_rand_score
from ipynb.fs.full.Cluster import cluster
from ipynb.fs.full.Subsequence_Distance import identifyshapelets
from ipynb.fs.full.Distance_Information import transform
from ipynb.fs.full.Label_Assignment import assign
from ipynb.fs.full.Change_Point import changepoint
from ipynb.fs.full.L_Calculation import cal_L
from ipynb.fs.full.Decision_Function import decision
from ipynb.fs.full.test_distance import trans

In [2]:
ts_df =pd.read_csv(r"D:\Dr. Sheng Li\Datasets\UCRArchive_2018\GunPoint\sample2.tsv",sep="\t")# Line 2 in table 2
dataset=ts_df.drop(ts_df.columns[0],axis=1)
dataset.columns = range(dataset.shape[1])
labels=ts_df['0']
labels=np.array(labels)
unique_labels=np.unique(labels)
num_classes=unique_labels.shape[0]
# labels
dataset.shape

(16, 12)

In [4]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.647885,-0.641992,-0.638186,-0.638259,-0.638345,-0.638697,-0.643049,-0.643768,-0.645050,-0.647118,-0.649153,-0.651246
1,-0.644427,-0.645401,-0.647055,-0.647492,-0.646910,-0.643884,-0.639731,-0.638094,-0.635297,-0.635384,-0.634114,-0.633719
2,-0.778353,-0.778279,-0.777151,-0.777684,-0.775900,-0.772421,-0.765464,-0.762275,-0.763752,-0.765356,-0.765617,-0.765978
3,-0.750060,-0.748103,-0.746164,-0.745926,-0.743767,-0.743805,-0.745213,-0.745082,-0.745727,-0.745815,-0.745989,-0.745682
4,-0.599539,-0.597422,-0.599269,-0.598259,-0.597582,-0.591303,-0.589020,-0.587533,-0.585462,-0.583848,-0.581602,-0.578487


In [5]:
shape=[10,15]#shapelet length
rep_df=cluster(shape,ts_df,labels)# From 'Cluster' notebook
Shapelets=[]
subsequence_sample=[10]# Length of shapelets in DTEC
rep_labels=rep_df[0]
rep=rep_df.drop(rep_df.columns[0],axis=1)#representative time series
rep_labels=rep_labels.astype('int')
Shapelets,Earliness=identifyshapelets(ts_df.copy(),rep,subsequence_sample,rep_labels)# From 'Subsequence_Distance' notebook
# Earliness
Earliness.insert(0,0)

uShapelets Identified
clustering of class 1 complete
uShapelets Identified
clustering of class 2 complete
*********************************
Shapelets Identified


In [6]:
rep_df.to_csv('test.csv',index=False)

In [13]:
D_list=[]
df=pd.DataFrame()
df1=pd.DataFrame()
global_chpoint=[]
global_nochpoint=[]
local_nochpoint=[]
local_chpoint=[]
timestamp=[]
D_org=[]
l=len(max(Shapelets,key=len))-1
for i in range(0,dataset.shape[0]):
    D_temp=transform(Shapelets,dataset.iloc[i],len(max(Shapelets,key=len)))# From 'Distance_information' notebook
    D,chpoint,nochpoint=changepoint(D_temp)
    df=pd.DataFrame(D)
    local_chpoint.append(chpoint)
    chpoint = [x+l for x in chpoint]
    local_nochpoint.append(nochpoint)
    nochpoint = [x+l for x in nochpoint]
    D_list.append(D)
    D_org.append(D)
    global_chpoint.append(chpoint)
    global_nochpoint.append(nochpoint)
    test=np.isfinite(D)
    for a in test:
        for value in a:
            if value == False:
                print('nan or inf detected')

In [14]:
flag=True
for k in range(0,len(D_list)):
    for i in range(0,D_list[0].shape[1]):
        for j in range(1,D_list[0].shape[0]-1):
            if D[j][i]>D[j-1][i]:
                flag=False
                break
        if flag==False:
            break
    if flag==False:
        break
flag

True

In [15]:
Y_org=[]
Y_list=[]#label list
for i in range(0, len(D_list)):
    Y=assign(dataset.iloc[i],labels[i],D_list[i],num_classes,global_chpoint[i],len(max(Shapelets,key=len)))#assigns label
    Y_org.append(Y)
    for j in range(0,(len(max(Shapelets,key=len))-1)):
        global_nochpoint[i].append(j)
    Y=np.delete(Y,global_nochpoint[i],0)
    test=np.isfinite(Y)
    for a in test:
        for value in a:
            if value == False:
                print('nan or inf detected')
#     Y=np.nan_to_num(Y,nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
    Y_list.append(Y)
for i in range(0,len(D_list)):
    D_list[i]=np.delete(D_list[i],local_nochpoint[i],0)

In [16]:
# for i in Y_list:
#     print(i)
#     print('####################')

In [17]:
# for i in range(0,len(Y_list)):
#     print('iteration',i)
#     print('Y',Y_list[i].shape)
#     print('D',D_list[i].shape)
#     print('********************************')

In [18]:
# for i in range(0,len(D_list)):
#     print('D shape',D_list[i].shape)
#     print('L shape',L_list[i].shape)
#     print('********************************')

In [19]:
# df=pd.DataFrame(D_org[5])
# df

In [20]:
L_list=[]
for i in range(0,len(Y_list)):
    L_temp=cal_L(Y_list[i],labels[i],num_classes)
    test=np.isfinite(L_temp)
    for a in test:
        for value in a:
            if value == False:
                print('nan or inf detected',i)
                L_temp=np.nan_to_num(L_temp,nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
    L_list.append(L_temp)

In [11]:
def fig(i,j,k,t):
    print('here')
    s='lambda1 = '+str(i)+" lambda2 = "+str(j)+" rho = "+str(k)
    y=np.arange(50)
    plt.scatter(y,W_list)
    plt.xlabel('Iterations')
    plt.ylabel('Difference')
    plt.text(9,W_list[0],s)
    ss='Figure'+str(t)+'.jpeg'
    plt.savefig(ss)
    plt.show()

In [15]:
rho_list=[0.001]#,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.02,0.03,0.04,0.05,0.06,0.07,0.08]
lambda1_list=[0.1]#,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
lambda2_list=[0.1,0.2]#,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

In [21]:
test=1
for i in lambda1_list:
    for j in lambda2_list:
        for k in rho_list:
            print('here')
            W_list=[]
            W,W_list=decision(dataset,D_list,Earliness,num_classes,L_list,len(Shapelets),50,i,j,k)
            fig(i,j,k,test)
            test=test+1

In [10]:
#     lambda1=.01#hyperparameter
#     lambda2=.01#hyperparameter
#     rho=.0001#hyperparameter
W,W_list=decisionn(dataset,D_list,Earliness,num_classes,L_list,len(Shapelets),5,0.01,0.01,0.001)

NameError: name 'decisionn' is not defined

In [23]:
#     lambda1=.01#hyperparameter
#     lambda2=.01#hyperparameter
#     rho=.0001#hyperparameter
W,W_list=decision(dataset,D_list,Earliness,num_classes,L_list,len(Shapelets),10,0.9,0.9,0.8)

0 iteration : 6.034465593686552
***************************************************************
1 iteration : 0.0034266008891469983
***************************************************************
2 iteration : 1.943969419232392e-06
***************************************************************
3 iteration : 1.102930648887832e-09
***************************************************************
4 iteration : 6.257589456730803e-13
***************************************************************
5 iteration : 3.5487976162877596e-16
***************************************************************
6 iteration : 4.599880250050123e-19
***************************************************************
7 iteration : 0.0
***************************************************************
8 iteration : 0.0
***************************************************************
9 iteration : 0.0
***************************************************************
Done


In [22]:
# def decisionn(df,Dlist,E,K,Llist,H,iterations,lambda1,lambda2,rho):
#     W_list=[]
#     W=np.random.rand(K-1,H+1)#different from paper
#     W_temp=np.random.rand(K-1,H+1)#different from paper
#     W_temp=W_temp/(np.linalg.norm(W_temp,'fro'))
#     A=np.random.rand(K-1,H+1)#different from paper
#     A=A/(np.linalg.norm(A,'fro'))
#     B=np.random.rand(K-1,H+1)#different from paper
#     B=B/(np.linalg.norm(B,'fro'))
#     C=np.random.rand(K-1,H+1)#different from paper
#     C=C/(np.linalg.norm(C,'fro'))
#     U=np.random.rand(K-1,H+1)
#     U=U/(np.linalg.norm(U,'fro'))
#     V=np.random.rand(K-1,H+1)
#     V=V/(np.linalg.norm(V,'fro'))
# #     lambda1=.01#hyperparameter
# #     lambda2=.001#hyperparameter
# #     rho=.0001#hyperparameter
#     ind=df.index.values
#     flag=True
#     for item in range(0,iterations):
#         W=calculateW(df,Llist,Dlist,A,U,B,V,ind,rho)
#         W=W/(np.linalg.norm(W,'fro'))
# #         flag=True
# #         for i in range(0,W.shape[0]):
# #             for j in range(0,W.shape[1]):
# #                 if W[i][j] == W_temp[i][j]:
# #                     flag=True
# #                 else:
# #                     flag=False
# #                     break
# #             if flag == False:
# #                 break
        
# #         print(W)
# #         print('WWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWW')
# #         print(U)
# # #         print('UUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUU')
#         A=calculateA(C,A,W,U,lambda2,E,rho)
# #         print(flag)
#         WU=W+U
# #         print(WU)
#         B=calculateB(W,V,WU,lambda1,rho)
# #         diff=W-W_temp
#         print(item,"iteration :",np.linalg.norm((W-W_temp),2))
#         W_list.append(np.linalg.norm((W-W_temp),2))
#         W_temp=W
#         print("***************************************************************")
#     print("Done")
#     return W,W_list

In [15]:
# def calculateW(df,Llist,D_list,A,U,B,V,index,rho):
#     for k in range(0,df.shape[0]):
#         ts=df.iloc[k]
#         Ln=Llist[k]
#         Ln=Ln.T
#         Dq=D_list[k]
# #         Dq=np.nan_to_num(Dq)
#         first=np.dot(Ln,Dq)+(0.5*rho*A)-(0.5*rho*U)+(0.5*rho*B)-(0.5*rho*V)
# #         print(first)
# #         print('#############################')
# #         first=np.nan_to_num(first,nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
#         second_one=np.dot(Dq.T,Dq)#+2*rho*(np.identity(Dq.shape[1])))
# #         second_one=np.nan_to_num(second_one,nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
#         second_second=2*rho*(np.identity(second_one.shape[0]))
# #         second_second=np.nan_to_num(second_second,nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
#         second_temp=second_one+second_second
# #         second_temp=np.nan_to_num(second_temp,nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
#         det = np.linalg.det(second_temp)
#         if det!=0:
#             second=linalg.inv(second_temp)
#         else:
#             print('here')
#             second=second_temp
# #         second=np.nan_to_num(second,nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
#         w=np.dot(first,second_temp)
# #         w=np.nan_to_num(w,nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
#     return w

In [16]:
# def calculateA(C,A,W,U,lambda2,E,rho):
#     j=0
#     i=0
#     while j<W.shape[1]:
#         C=W[:,i]+U[:,i]
#         temp1=rho*np.linalg.norm(C,2)
#         temp2=2*lambda2*E[i]
#         temp=temp2/temp1
#         temp=1-temp
#         if temp<=0:
#             A[:,i]=0
#         else:
#             A[:,i]=temp
# #             print('greater than')
# #         print('temp',temp)
# #         A[:,i]=temp
# #         if A[:,i].all()<=0:
# #             A[:,i]=0
# #         A[:,i]=np.nan_to_num(A[:,i],nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
#         i=i+1
#         j=j+1
#     return A

In [17]:
# def calculateB(W,V,WU,lambda1,rho):
#     b=np.empty([W.shape[0],W.shape[1]])
#     for i in range(0,W.shape[0]):
#         for j in range(0,W.shape[1]):
#             if WU[i][j]>=((1-lambda1)/rho):
#                 b[i][j]=(W[i][j]+V[i][j])-((1-lambda1)/rho)
#             elif WU[i][j]<=(-1)*((1-lambda1)/rho):
#                 b[i][j]= (W[i][j]+V[i][j])+((1-lambda1)/rho)
#             else:
#                 b[i][j]=0
# #     b=np.nan_to_num(b,nan=random.uniform(0,1),posinf=random.uniform(0,1),neginf=random.uniform(0,1))
#     return b

In [38]:
test_df =pd.read_csv(r"D:\Dr. Sheng Li\Datasets\UCRArchive_2018\CBF\sample1test.tsv",sep="\t")
test_data=test_df.drop(test_df.columns[0],axis=1)
test_labels=test_df['0']
test_labels=np.array(test_labels)
length=len(max(Shapelets,key=len))
test_labels=test_labels-1

In [48]:
test_labels

array([1, 1, 0, 2, 0, 2], dtype=int64)

In [42]:
def predict(W,D):
    Pr_K=0
    Dt=D.T
    sum=0
    for i in range(0,W.shape[0]):
        first=W[i][0]
        second_temp=W[i][1:W.shape[1]]
        second=np.dot(second_temp,Dt)
#         print(first+second)
        third=np.exp(first+second)
        third=np.ndarray.item(third)
        sum=sum+(third)
#     print(sum)
    sum=1+sum
    Pr_K=(1/sum)
#     print(Pr_K)
    prob=np.zeros((W.shape[0]+1))
    prob[-1]=Pr_K
    for i in range(0,W.shape[0]-1):
        first=W[i][0]
        second_temp=W[i][1:W.shape[1]]
        second=np.dot(second_temp,Dt)
#         print(first+second)
        third=np.exp(first+second)
        third=np.ndarray.item(third)
        prob[i]=Pr_K*third
    return prob

In [43]:
predicted_labels=[]
threshold=6
timestamp=[]
df=pd.DataFrame()
for i in range(0,test_data.shape[0]):
    temp=test_data.iloc[i]
    temp=list(temp)
    c_area=0
    stamp=0
    for j in range(1, len(temp)+1):
        stamp=j-1
        ts=temp[0:j]
        D_temp=trans(Shapelets,ts)
#         df=pd.DataFrame(D_temp)
        proba=predict(W,D_temp)
        sorted_prob=-np.sort(-proba)
#         print('iteration :',j)
#         print('initial probability ',proba)
#         print('sorted prob ',sorted_prob)
#         print('###################################')
        ind=np.argsort(-proba)
#         print(ind)
        if j==1:
            prev_ind=ind[0]
#             print('in j==1')
            c_area=sorted_prob[0]-sorted_prob[1]
#             print(c_area)
            if c_area>threshold:
#                 print('first')
                predicted_labels.append(ind[0])
                break
        elif prev_ind == ind[0]:
#             print('in next')
            c_area=c_area+(sorted_prob[0]-sorted_prob[1])
#             print(c_area)
            if c_area>threshold:
#                 print('second')
                predicted_labels.append(ind[0])
                break
        elif prev_ind != ind[0]:
#             print('in last')
            prev_ind=ind[0]
            c_area=sorted_prob[0]-sorted_prob[1]
#             print(c_area)
            if c_area>threshold:
#                 print('third')
                predicted_labels.append(ind[0])
                break
#         print('#####')
    timestamp.append(stamp)

In [44]:
predicted_labels

[0, 0, 0, 0, 0, 0]

In [45]:
timestamp

[7, 7, 7, 7, 7, 7]

In [46]:
Early=0.0
total=0.0
for i in range(0,test_data.shape[0]):
    total=total+(timestamp[i]/test_data.shape[1])
Early=total/test_data.shape[0]

In [55]:
Early

0.35000000000000003

In [50]:
Error_rate=0.0
incorrect=0
for i in range(0,test_data.shape[0]):
    if test_labels[i]!=predicted_labels[i]:
        incorrect=incorrect+1
Error_rate=incorrect/test_data.shape[0]

In [51]:
Error_rate

0.6666666666666666

In [53]:
alpha=0.5
F_alpha=1/(((1-alpha)/Error_rate)+(alpha/Early))

In [54]:
F_alpha

0.459016393442623